In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pandas.io import sql
import sqlite3

/Users/michaeltwardos/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [9]:
conn = sqlite3.connect('scaledown.db', detect_types=sqlite3.PARSE_DECLTYPES)

In [11]:
users = pd.read_csv('sd_users.csv', low_memory=False)
users.head()

,id,signed_up_on,email_confirmed_at,sms_confirmed_at,created_at,updated_at,time_zone,gender,age,zip_code,...,sms_reminders,scale_sent_on,sales_rep,lat,lng,carrier_name,carrier_type,report_slug,ivp_week,affiliate_id
0,70,NaN,NaN,NaN,2015-03-10 03:47:38,2016-01-06 06:26:07,NaN,NaN,NaN,60115,...,1,NaN,0,NaN,NaN,NaN,NaN,976254S06,NaN,NaN
1,71,NaN,NaN,NaN,2015-03-10 03:47:39,2016-01-06 06:28:13,NaN,Male,32,95014,...,1,NaN,0,NaN,NaN,NaN,NaN,R73652875,NaN,NaN
2,72,NaN,NaN,NaN,2015-03-10 03:47:39,2016-01-06 06:27:48,NaN,Female,30,95014,...,1,NaN,0,NaN,NaN,NaN,NaN,125H62474,NaN,NaN
3,73,NaN,NaN,NaN,2015-03-10 03:47:39,2015-06-04 15:12:43,NaN,NaN,NaN,60115,...,1,NaN,0,NaN,NaN,verizon wireless,mobile,38599404G,NaN,NaN
4,74,2015-04-20,NaN,NaN,2015-03-10 03:47:56,2015-06-04 15:12:43,NaN,NaN,NaN,60115,...,1,NaN,0,NaN,NaN,google (grand central) bwi,voip,9W7876474,NaN,NaN


In [4]:
weights = pd.read_csv('sd_weights.csv', low_memory=False, skiprows=7000000, nrows=1000000)
len(weights)

206411

In [17]:
#There are 7,206,411 rows in weights
weights = pd.read_csv('sd_weights.csv', low_memory=False,  nrows=1000000)
weights.head(3)

,id,user_id,grams,lbs,recorded_at,created_at,updated_at,weighing_time,status,tare,reasoning,source_id,source_type
0,71,87,117900,259.9,2015-03-26 19:10:15,2015-03-26 19:10:23,2015-03-26 19:10:23,1427397015,0,NaN,NaN,NaN,NaN
1,72,85,73000,160.9,2015-03-27 10:36:43,2015-03-27 10:36:46,2015-03-27 10:36:46,1427452603,0,NaN,NaN,NaN,NaN
2,73,88,98700,217.6,2015-03-27 10:45:53,2015-03-27 10:45:55,2015-03-27 10:45:55,1427453153,0,NaN,NaN,NaN,NaN


In [19]:
weights.to_sql('weights',
            con=conn,
            if_exists='replace',
            index=False)

In [12]:
users.to_sql('users',
            con=conn,
            if_exists='replace',
            index=False)

In [16]:
a=sql.read_sql('''
select
    a.user_id,
    a.first_weight,
    count(*) as weights
from
(
select  
    user_id, 
    min(weighing_time) as first_weight
from 
    weights
group by 1) a
left join
    weights b
on 
    a.user_id = b.user_id
where
    b.weighing_time - a.first_weight between 0 and 30*86400
group by 1, 2 limit 20
;
''', con=conn)
a.head()

,user_id,first_weight,weights
0,84,1427483216,57
1,85,1427452603,23
2,86,1427453400,26
3,87,1427397015,19
4,88,1427453153,23
5,89,1427843853,13
6,90,1427822475,29
7,92,1427709411,30
8,93,1427887611,21
9,95,1428105496,39


In [26]:
a=sql.read_sql('''
select
    c.user_id,
    c.first_weight_time,
    c.first_weight,
    count(*) as first_month_measurements
from
(select
    a.user_id,
    a.first_weight as first_weight_time,
    b.lbs as first_weight
from
(
select  
    user_id, 
    min(weighing_time) as first_weight
from 
    weights
group by 1
) a
left join
    weights b
on 
    a.user_id = b.user_id and
    a.first_weight = b.weighing_time
group by 1, 2
) c
left join
    weights d
on
    c.user_id = d.user_id
where
    d.weighing_time - c.first_weight_time between 0 and 30*86400
group by 1, 2, 3
;
''', con=conn)
a.head()

,user_id,first_weight_time,first_weight,first_month_measurements
0,84,1427483216,161.2,57
1,85,1427452603,160.9,23
2,86,1427453400,208.3,26
3,87,1427397015,259.9,19
4,88,1427453153,217.6,23


In [27]:
a=sql.read_sql('''

select
    e.user_id,
    e.first_weight_time,
    e.first_weight,
    e.first_month_measurements,
    avg(lbs) as average_old_weight
from
(select
    c.user_id,
    c.first_weight_time,
    c.first_weight,
    count(*) as first_month_measurements
from
(select
    a.user_id,
    a.first_weight as first_weight_time,
    b.lbs as first_weight
from
(
select  
    user_id, 
    min(weighing_time) as first_weight
from 
    weights
group by 1
) a
left join
    weights b
on 
    a.user_id = b.user_id and
    a.first_weight = b.weighing_time
group by 1, 2
) c
left join
    weights d
on
    c.user_id = d.user_id
where
    d.weighing_time - c.first_weight_time between 0 and 30*86400
group by 1, 2, 3
) e
left join
    weights f
on
    e.user_id = f.user_id
where
    f.weighing_time - e.first_weight_time between 60*86400 and 90*86400 
group by 1, 2, 3, 4
;
''', con=conn)
a.head()

,user_id,first_weight_time,first_weight,first_month_measurements,average_old_weight
0,84,1427483216,161.2,57,147.086111
1,85,1427452603,160.9,23,154.694444
2,86,1427453400,208.3,26,206.826923
3,87,1427397015,259.9,19,224.283333
4,88,1427453153,217.6,23,181.143750


In [29]:
a.to_csv('results.csv')